### 2. Pdf naar txt

Het doel van dit script is om de inhoud van pdf-bestanden te kopïeren naar txt-bestanden. Hierbij wordt gebruik gemaakt van Optical Character Recognition (OCR) met behulp van de pytesseract applicatie. Dit script behoeft de volgende installaties, die in de prompt met de onderstaande commandos geïnstalleerd kunnen worden:
 
- `pip install pytesseract `
- `pip install Pillow`
- `pip install pdf2image`

Documentatie:

- Pytesseract: https://pypi.org/project/pytesseract/
- Pillow: https://pypi.org/project/Pillow/
- PDF2Image: https://pypi.org/project/pdf2image/

Daarnaast kan ook een Nederlands talenmodel worden gebruikt om de kwaliteit van de OCR te verbeteren: tesseract-ocr-nld. Download de Nederlandse trainingsdata hier https://github.com/tesseract-ocr/tessdata_best en plaats het in C:\\Program Files\\Tesseract-OCR\\tessdata (Windows)


#### Stap 1: Pdf-bestanden uit meerdere mappen verzamelen in aparte map (optioneel)

Het onderstaande celblokje maakt het mogelijk om alle pdf-bestanden binnen een hoofdmap en meerdere submappen te vinden en te verplaatsen naar een andere map. Met behulp van de glob module kunnen objectnamen worden opgehaald op basis van een bepaald patroon, in dit geval bestanden die eindigen met .pdf. De shutil module zorgt ervoor dat deze bestanden gekopieerd worden naar een andere map. Zorg ervoor dat er eerst een nieuwe map is aangemaakt voor de gekopieerde pdf-bestanden. Mocht je al een map hebben met alle pdf-bestanden, kan je deze stap overslaan en verder gaan met stap 2.

Documentatie: 
- glob documentatie https://docs.python.org/3/library/glob.html (let op: glob haalt geen padnamen op van bestanden uit gezipte mappen)
- shutil documentatie https://docs.python.org/3/library/shutil.html 

Let op de opmaak bij het invullen en kopiëren van het pad:

- fout: 'C:\Users\User\Documents\Voorbeeld\'
- goed: 'C:/Users/User/Documents/Voorbeeld/'


In [ ]:
import glob
from glob import glob
import os
import shutil

path = '' + '/' #VUL IN: plaats tussen de eerste aanhalingstekens het pad naar de map waar je bestanden staan

pdf = glob(os.path.join(path, '**/*.pdf'), recursive = True)                                

dst_folder = '' + '/' #VUL IN: plaats tusen de eerste aanhalingstekens het pad naar de map waarnaar de pdf-bestanden worden gekopieerd 
                                            
for file in pdf:
    file_name = os.path.basename(file)
    shutil.copy(file, dst_folder + file_name)
    print('Gekopieeerd naar:', file) #lijst geprint met gekopieerde pdf-bestanden

#### Stap 2: Pdf-bestanden omzetten naar txt:

In het onderstaande celblokje worden de pdf-bestanden met behulp van pytesseract machine leesbaar gemaakt. Pytesseract maakt een pdf-bestand leesbaar door van elk pdf-pagina een afbeelding te maken, de tekst op de afbeelding te lezen, en vervolgens te kopiëren naar een txt-bestand. Afhankelijk van de de lengte en de hoeveelheid van de pdf-bestanden kan het meerdere uren duren voordat alle pdf-bestanden zijn omgezet. 

Zoals te zien is in het onderstaande celblokje is os.listdir gevoelig voor hoofdletters. Daarom wordt zowel op .pdf als .PDF gezocht. Mocht er een andere spellingsvariant tussen je bestanden zitten, voeg dan de volgende twee lijnen toe aan het celblokje met de andere spellingsvariant:
```python
elif pdf.endswith('[.PdF]'): #voeg tussen de vierkante haakjes de variant toe en haal de [] weg
    index.append(pdf)
```

Een veelvoorkomende error is TesseractNotFound. Deze error kan eventueel worden opgelost door in het onderstaande celblokje de hashtag (#) weg te halen voor:
```python
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract'
```
Let op de opmaak bij het invullen en kopiëren van het pad:

- fout: 'C:\Users\User\Documents\Voorbeeld\'
- goed: 'C:/Users/User/Documents/Voorbeeld/'

In [ ]:
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path
import os
import glob

#pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract' #VUL IN: plaats tussen de aanhalingstekens het pad naar het executable bestand van tesseract. Voorbeeld: C:/Program Files/Tesseract-OCR/tesseract

os.chdir(dst_folder)  
                                                                             
pdfs = [] 

for pdf in os.listdir(dst_folder):
    if pdf.endswith('.pdf'):  
        pdfs.append(pdf)
    elif pdf.endswith('.PDF'):
        pdfs.append(pdf)

for pdf in pdfs:
    pages = convert_from_path(pdf, 500) #500 staat voor dots per inch (dpi), waarschijnlijk is 300 ook voldoende. 
                                        #300 dpi kan het script wellicht ook sneller laten draaien
    
    image_counter = 1
    
    for page in pages:
        filename = f"{pdf.title()}"+str(image_counter)+".jpg" 
        page.save(filename, 'JPEG')
        image_counter = image_counter + 1
        
        filelimit = image_counter-1
        
        outfile = f"{pdf.title()}.txt" 
        
        f = open(outfile, "a")
        
    for i in range(1, filelimit + 1):
            filename = f"{pdf.title()}"+str(i)+".jpg"
            text = str(((pytesseract.image_to_string(Image.open(filename), lang='nld'))))  
            text = text.replace('-\n', '')    
            f.write(text)
    f.close()

#### Stap 3: Afbeeldingen verwijderen uit map

Voor elke pdf-pagina is een afbeelding (.jpg) gemaakt. Deze hoeven niet bewaard te worden voor de rest van de Topic Modeling scripts en kunnen met het onderstaande celblokje worden verwijderd uit de map met pdf-bestanden.

In [ ]:
#verwijderen van alle .jpg bestanden uit de map met pdfs: 
import glob
from glob import glob

fileList = glob(os.path.join(dst_folder, '*.jpg'))   

for filePath in fileList:                           
    try:
        os.remove(filePath)
    except OSError:
        print("Error tijdens het verwijderen van de bestanden")

#### Einde

Als het goed is, staat nu voor ieder pdf-bestand in de map ook een txt-bestand.

- Ga verder met het script *Docx naar txt* als er ook docx-bestanden in je archief zitten.
- Ga verder met het script *Corpus samenstellen* om een corpus samen te stellen met docx- en/of pdf-bestanden